# Locks

## Lock()

In [1]:
from threading import Lock, Thread, current_thread
import time


class SampleLocks:
    def __init__(self):
        self.value = 0
        self.lock = Lock()
        
    def long_lock(self):
        with self.lock:
            print(f"Insiude with self.lock. Before sleep: {current_thread().name = }, {self.lock.locked() = }")
            time.sleep(5) # long operation
            print(f"Insiude with self.lock. After sleep: {current_thread().name = }, {self.lock.locked() = }")
        
        print(f"Outside self.lock.: {current_thread().name = }, {self.lock.locked() = }")

sl = SampleLocks()
sl.long_lock()

Insiude with self.lock. Before sleep: current_thread().name = 'MainThread', self.lock.locked() = True
Insiude with self.lock. After sleep: current_thread().name = 'MainThread', self.lock.locked() = True
Outside self.lock.: current_thread().name = 'MainThread', self.lock.locked() = False


In [2]:
from threading import Lock, Thread, current_thread
import time



class SampleLocks:
    def __init__(self):
        self.value = 0
        self.lock = Lock()
        
    def long_lock(self):
        with self.lock:
            print(f"{current_thread().name = }: Inside `with self.lock`. Before sleep. {self.lock.locked() = }")
            time.sleep(5)
            print(f"{current_thread().name = }: Inside `with self.lock`. After sleep. {self.lock.locked() = }")
        
        # Andrey: Add comment
        print(f"{current_thread().name = }: Outside self.lock. {self.lock.locked() = }")


sl = SampleLocks()
tr = Thread(target=sl.long_lock)
tr.start()

time.sleep(2)  # long operation but less than in SampleLocks.long_lock()
print(f"{current_thread().name = }: After lock in other thread. {sl.lock.locked() = }")

sl.lock.release()
print(f"{current_thread().name = }: Other Thread after release lock. {sl.lock.locked() = }")

current_thread().name = 'Thread-4': Inside `with self.lock`. Before sleep. self.lock.locked() = True
current_thread().name = 'MainThread': After lock in other thread. sl.lock.locked() = True
current_thread().name = 'MainThread': Other Thread after release lock. sl.lock.locked() = False


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-cf365705e3b7>", line 15, in long_lock
RuntimeError: release unlocked lock


current_thread().name = 'Thread-4': Inside `with self.lock`. After sleep. self.lock.locked() = False


In [3]:
from threading import Lock, Thread, current_thread
import time



class SampleLocks:
    def __init__(self):
        self.value = 0
        self.lock = Lock()
        
    def first_lock(self):
        print(f"{current_thread().name = }: First lock try acquire. {self.lock.locked() = }")
        self.lock.acquire() # lock first time
        print(f"{current_thread().name = }: First lock acquired. {self.lock.locked() = }")
        
        self.second_lock() # lock second time
            
            
    def second_lock(self):
        print(f"{current_thread().name = }: Second lock try acquire. {self.lock.locked() = }")
        self.lock.acquire()
        print(f"{current_thread().name = }: Second lock acquired? {self.lock.locked() = }")
            
sl = SampleLocks()
tr = Thread(target=sl.first_lock)
tr.start()


time.sleep(1) # Sleep for sure that lock acquired
print(f"{current_thread().name = }: Other Thread after some time. {sl.lock.locked() = }")
sl.lock.release()
print(f"{current_thread().name = }: Other Thread after release lock. {sl.lock.locked() = }")


time.sleep(1) # Sleep for sure that lock acquired... again
print(f"{current_thread().name = }: Other Thread after sometime in other thread. {sl.lock.locked() = }")

current_thread().name = 'Thread-5': First lock try acquire. self.lock.locked() = False
current_thread().name = 'Thread-5': First lock acquired. self.lock.locked() = True
current_thread().name = 'Thread-5': Second lock try acquire. self.lock.locked() = True
current_thread().name = 'MainThread': Other Thread after some time. sl.lock.locked() = True
current_thread().name = 'MainThread': Other Thread after release lock. sl.lock.locked() = False
current_thread().name = 'Thread-5': Second lock acquired? self.lock.locked() = True
current_thread().name = 'MainThread': Other Thread after sometime in other thread. sl.lock.locked() = True


## RLock()

In [5]:
from threading import RLock, Thread, current_thread
import time


class SampleLocks:
    def __init__(self):
        self.value = 0
        self.lock = RLock()  # Now use RLock
        
    def first_lock(self):
        print(f"{current_thread().name = }: Try acquire lock. {self.lock!r}")
        with self.lock:  # lock first time
            print(f"{current_thread().name = }: Lock acquired. {self.lock!r}")
            self.second_lock() # lock second time
        print(f"{current_thread().name = }: Lock released?. {self.lock!r}")
            
    def second_lock(self):
        print(f"{current_thread().name = }: Try acquire lock same lock again. {self.lock!r}")
        with self.lock:  # lock first time
            print(f"{current_thread().name = }: Same lock acquired again? {self.lock!r}")
            time.sleep(5)
        print(f"{current_thread().name = }: Same lock released?. {self.lock!r}")


sl = SampleLocks()
tr = Thread(target=sl.first_lock)
tr.start()


time.sleep(1) # Sleep for sure that lock acquired
print(f"{current_thread().name = }: Other Thread after some time. {sl.lock!r}")

with sl.lock:
    print(f"{current_thread().name = }: Other Thread use lock. {sl.lock!r}")
    time.sleep(5)
    print(f"{current_thread().name = }: Other Thread use lock. {sl.lock!r}")


current_thread().name = 'Thread-7': Try acquire lock. <unlocked _thread.RLock object owner=0 count=0 at 0x7f01ac278840>
current_thread().name = 'Thread-7': Lock acquired. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac278840>
current_thread().name = 'Thread-7': Try acquire lock same lock again. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac278840>
current_thread().name = 'Thread-7': Same lock acquired again? <locked _thread.RLock object owner=139644813436672 count=2 at 0x7f01ac278840>
current_thread().name = 'MainThread': Other Thread after some time. <locked _thread.RLock object owner=139644813436672 count=2 at 0x7f01ac278840>
current_thread().name = 'Thread-7': Same lock released?. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac278840>
current_thread().name = 'Thread-7': Lock released?. <unlocked _thread.RLock object owner=0 count=0 at 0x7f01ac278840>current_thread().name = 'MainThread': Other Thread use lock. <loc

In [6]:
from threading import RLock, Thread, current_thread
import time


class SampleLocks:
    def __init__(self):
        self.value = 0
        self.lock = RLock()  # Now use RLock
        
    def first_lock(self):
        print(f"{current_thread().name = }: Try acquire lock. {self.lock!r}")
        with self.lock:  # lock first time
            print(f"{current_thread().name = }: Lock acquired. {self.lock!r}")
            self.second_lock() # lock second time
        print(f"{current_thread().name = }: Lock released?. {self.lock!r}")
            
    def second_lock(self):
        print(f"{current_thread().name = }: Try acquire lock same lock again. {self.lock!r}")
        with self.lock:  # lock first time
            print(f"{current_thread().name = }: Same lock acquired again? {self.lock!r}")
            time.sleep(5)
        print(f"{current_thread().name = }: Same lock released?. {self.lock!r}")


sl = SampleLocks()
tr = Thread(target=sl.first_lock)
tr.start()


time.sleep(1) # Sleep for sure that lock acquired
print(f"{current_thread().name = }: Other Thread after some time. {sl.lock!r}")

sl.lock.release()  # We can't release RLock which acquired by another Thread

current_thread().name = 'Thread-8': Try acquire lock. <unlocked _thread.RLock object owner=0 count=0 at 0x7f01ac2c53f0>
current_thread().name = 'Thread-8': Lock acquired. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac2c53f0>
current_thread().name = 'Thread-8': Try acquire lock same lock again. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac2c53f0>
current_thread().name = 'Thread-8': Same lock acquired again? <locked _thread.RLock object owner=139644813436672 count=2 at 0x7f01ac2c53f0>
current_thread().name = 'MainThread': Other Thread after some time. <locked _thread.RLock object owner=139644813436672 count=2 at 0x7f01ac2c53f0>


RuntimeError: cannot release un-acquired lock

current_thread().name = 'Thread-8': Same lock released?. <locked _thread.RLock object owner=139644813436672 count=1 at 0x7f01ac2c53f0>
current_thread().name = 'Thread-8': Lock released?. <unlocked _thread.RLock object owner=0 count=0 at 0x7f01ac2c53f0>


# multiprocessing

## `__name__ == "__main__"`: fork vs spawn

__in Jupyter we can't simulate this behaviour__



```python
import multiprocessing as mp


def dummy():
    print("Dummy")


context = mp.get_context("fork")
process = context.Process(target=dummy)
process.start()
process.join()
```

__Output__

```shell-session
Dummy
```

__in Jupyter we can't simulate this behaviour__



```python
import multiprocessing as mp


def dummy():
    print("Dummy")


context = mp.get_context("spawn")
process = context.Process(target=dummy)
process.start()
process.join()
```

__Output__

```py

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 125, in _main
    prepare(preparation_data)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 236, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 287, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
  File "/usr/lib/python3.9/runpy.py", line 268, in run_path
    return _run_module_code(code, init_globals, run_name,
  File "/usr/lib/python3.9/runpy.py", line 97, in _run_module_code
    _run_code(code, mod_globals, init_globals,
  File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/student/.config/JetBrains/PyCharmCE2020.3/scratches/scratch.py", line 10, in <module>
    process.start()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 121, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.9/multiprocessing/context.py", line 284, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_spawn_posix.py", line 32, in __init__
    super().__init__(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_spawn_posix.py", line 42, in _launch
    prep_data = spawn.get_preparation_data(process_obj._name)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 154, in get_preparation_data
    _check_not_importing_main()
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 134, in _check_not_importing_main
    raise RuntimeError('''
RuntimeError: 
        An attempt has been made to start a new process before the
        current process has finished its bootstrapping phase.

        This probably means that you are not using fork to start your
        child processes and you have forgotten to use the proper idiom
        in the main module:

            if __name__ == '__main__':
                freeze_support()
                ...

        The "freeze_support()" line can be omitted if the program
        is not going to be frozen to produce an executable.
```

__in Jupyter we can't simulate this behaviour__


```python
import multiprocessing as mp


def dummy():
    print("Dummy")


context = mp.get_context("forkserver")
process = context.Process(target=dummy)
process.start()
process.join()
```

__Output__


```py
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/forkserver.py", line 274, in main
    code = _serve_one(child_r, fds,
  File "/usr/lib/python3.9/multiprocessing/forkserver.py", line 313, in _serve_one
    code = spawn._main(child_r, parent_sentinel)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 125, in _main
    prepare(preparation_data)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 236, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 287, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
  File "/usr/lib/python3.9/runpy.py", line 268, in run_path
    return _run_module_code(code, init_globals, run_name,
  File "/usr/lib/python3.9/runpy.py", line 97, in _run_module_code
    _run_code(code, mod_globals, init_globals,
  File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/student/.config/JetBrains/PyCharmCE2020.3/scratches/scratch.py", line 10, in <module>
    process.start()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 121, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.9/multiprocessing/context.py", line 291, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_forkserver.py", line 35, in __init__
    super().__init__(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.9/multiprocessing/popen_forkserver.py", line 42, in _launch
    prep_data = spawn.get_preparation_data(process_obj._name)
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 154, in get_preparation_data
    _check_not_importing_main()
  File "/usr/lib/python3.9/multiprocessing/spawn.py", line 134, in _check_not_importing_main
    raise RuntimeError('''
RuntimeError: 
        An attempt has been made to start a new process before the
        current process has finished its bootstrapping phase.

        This probably means that you are not using fork to start your
        child processes and you have forgotten to use the proper idiom
        in the main module:

            if __name__ == '__main__':
                freeze_support()
                ...

        The "freeze_support()" line can be omitted if the program
        is not going to be frozen to produce an executable.

Process finished with exit code 0

```

__in Jupyter we can't simulate this behaviour__

```python
import multiprocessing as mp


def dummy():
    print("Dummy")


if __name__ == "__main__":
    context = mp.get_context("spawn")
#     context = mp.get_context("forkserver")
#     context = mp.get_context("fork")
    process = context.Process(target=dummy)
    process.start()
    process.join()
```

__Output__

```shell-session
Dummy
```

## multiprocessing.dummy

In [1]:
import threading
import os
from functools import wraps
import time


def thread_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"{threading.current_thread().name}. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"{threading.current_thread().name}. End function: {fn.__name__}")

    return wrapper


def process_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"Process ID: {os.getpid()}. Parrent Process: {os.getppid()}. {__name__ = }. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"Process ID: {os.getpid()}. End function: {fn.__name__}")

    return wrapper

In [8]:
import multiprocessing as mp
import multiprocessing.dummy as mp_dummy

In [9]:
help(mp_dummy.Pool)

Help on function Pool in module multiprocessing.dummy:

Pool(processes=None, initializer=None, initargs=())



In [10]:
dummy_mp_pool = mp_dummy.Pool()

In [12]:
type(dummy_mp_pool)

multiprocessing.pool.ThreadPool

In [13]:
mp_pool = mp.Pool()

In [14]:
type(mp_pool)

multiprocessing.pool.Pool

In [16]:
# Print MRO of class multiprocessing.dummy.Pool
mp.pool.ThreadPool.mro()

[multiprocessing.pool.ThreadPool, multiprocessing.pool.Pool, object]

In [17]:
# Print MRO of class multiprocessing.Pool
mp.pool.Pool.mro()

[multiprocessing.pool.Pool, object]

In [2]:
# Pool based on Processes

import multiprocessing as mp


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with mp.Pool(processes=3) as pool:
        result = pool.imap_unordered(squares, range(10))  # Doesn't guaranties order
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        for x in result:
            print(x)

Process ID: 15072. Parrent Process: 15059. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}Process ID: 15073. Parrent Process: 15059. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}Process ID: 15074. Parrent Process: 15059. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}


MainThread. Start function: squares. args = (1,), kwargs = {}MainThread. Start function: squares. args = (0,), kwargs = {}
MainThread. End function: squares
Process ID: 15073. End function: squares
Process ID: 15073. Parrent Process: 15059. __name__ = '__main__'. Start function: squares. args = (3,), kwargs = {}
MainThread. Start function: squares. args = (3,), kwargs = {}

MainThread. End function: squaresMainThread. End function: squares

Process ID: 15073. End function: squaresMainThread. Start function: squares. args = (2,), kwargs = {}Process ID: 15072. End function: squares
Process ID: 15073. Parrent Process: 15059. __name__ = '__main__'.

In [3]:
# Pool based on Threads
from multiprocessing.dummy import Pool as ThreadPool


@process_info
@thread_info
def squares(x):
    return x ** 2


if __name__ == '__main__':
    
    with ThreadPool(processes=3) as pool:
        result = pool.imap_unordered(squares, range(10))  # Doesn't guaranties order
        print(f"\n\nComplete. PID: {os.getpid()}. Result = {result}, {type(result)}")
        
        for x in result:
            print(x)




Complete. PID: 15059. Result = <multiprocessing.pool.IMapUnorderedIterator object at 0x7fe930b8af40>, <class 'multiprocessing.pool.IMapUnorderedIterator'>Process ID: 15059. Parrent Process: 3030. __name__ = '__main__'. Start function: squares. args = (0,), kwargs = {}
Thread-7. Start function: squares. args = (0,), kwargs = {}
Thread-7. End function: squares
Process ID: 15059. End function: squares
Process ID: 15059. Parrent Process: 3030. __name__ = '__main__'. Start function: squares. args = (1,), kwargs = {}
Thread-7. Start function: squares. args = (1,), kwargs = {}
Thread-7. End function: squares
Process ID: 15059. End function: squares
Process ID: 15059. Parrent Process: 3030. __name__ = '__main__'. Start function: squares. args = (2,), kwargs = {}
Thread-7. Start function: squares. args = (2,), kwargs = {}
Thread-7. End function: squares
Process ID: 15059. End function: squares
Process ID: 15059. Parrent Process: 3030. __name__ = '__main__'. Start function: squares. args = (3,

## Multiprocessing Programming guidelines
https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming

## Daemon=True

__in Jupyter we can't simulate this behaviour__

```python
import threading
import time

@thread_info
def dummy():
    while True:
        print("Dummy")
        time.sleep(1)


if __name__ == "__main__":
    worker = threading.Thread(target=dummy, daemon=True)
    worker.start()
    time.sleep(2)
```

__Output__

```shell-session
Dummy
Dummy
```

In [4]:
import threading

threading.enumerate()

[<_MainThread(MainThread, started 140639647999808)>,
 <Thread(Thread-2, started daemon 140639563785984)>,
 <Heartbeat(Thread-3, started daemon 140639555393280)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140639529953024)>,
 <ParentPollerUnix(Thread-1, started daemon 140639521298176)>]